# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
df = pd.read_csv("data/AviationData.csv", encoding="latin-1")
df["Make"] = df["Make"].str.strip().str.title()
df["Model"] = df["Model"].str.strip()
df.tail()

/var/folders/68/b1_q8gn50y50h0yyqy3025zc0000gn/T/ipykernel_12843/3766476407.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/AviationData.csv", encoding="latin-1")


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88888,20221230106513,Accident,ERA23LA097,2022-12-29,"Athens, GA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,30-12-2022


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [3]:
df["Event.Date"] = pd.to_datetime(df["Event.Date"])
df = df[(df["Event.Date"].dt.year >= 1983) & (df["Aircraft.Category"] == "Airplane")].copy()

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [4]:
df["Total.Passengers"] = (
      df["Total.Fatal.Injuries"].fillna(0) +
      df["Total.Serious.Injuries"].fillna(0) +
      df["Total.Minor.Injuries"].fillna(0) +
      df["Total.Uninjured"].fillna(0)
  )
df["Injury.Rate"] = (
        (df["Total.Fatal.Injuries"].fillna(0) + df["Total.Serious.Injuries"].fillna(0))
      / df["Total.Passengers"]
  )
#Assumptions have been made - that NAN in data is 0 

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [5]:
df = df[df["Aircraft.damage"].notna()]
df = df[df["Aircraft.damage"] != "Unknown"]
#take out any NAN and unknowns for better clarity
df["Is.Destroyed"] = (df["Aircraft.damage"] == "Destroyed").astype(int)

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [6]:
df = df[df["Make"].notna()]
df = df[df["Make"] != "Unknown"]
#take out NAN and unknown from the data for better clarity
df["Make"].value_counts()
make_counts = df["Make"].value_counts()
custom_makes = make_counts[make_counts >= 50].index
df= df[df["Make"].isin(custom_makes)]

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [7]:
df= df[df["Model"].notna()]
df= df[df["Model"] != "Unknown"]
#take out any NAN or unknown for better clarity
df.groupby("Model")["Make"].nunique().sort_values(ascending = False)
df["Make.Model"]= df["Make"] + "  " + df["Model"]

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [8]:
df["Engine.Type"] = df["Engine.Type"].replace({"UNK": "Unknown", "LR": "Unknown", "NONE": "Unknown"})
#clean up unknown values of engine type
df["Weather.Condition"] = df["Weather.Condition"].replace({"UNK": "Unknown", "Unk": "Unknown"})
#clean up weather conditions so similar vlaues are consolidated
df = df[df["Number.of.Engines"] != 0]
#clean up bad data where 0 was entered
df["Purpose.of.flight"] = df["Purpose.of.flight"].replace({
      "Air Race show": "Air Race/Show",
      "Air Race/show": "Air Race/Show",
      "PUBS": "Public Aircraft - State",
      "PUBL": "Public Aircraft - Local",
      "ASHO": "Air Race/Show"
  })
#clean up so similar types of flights are grouped together
df["Broad.phase.of.flight"] = df["Broad.phase.of.flight"].replace({"Unknown": np.nan})
# clean up by replacing the unknowns 

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [9]:
df.isnull().sum().sort_values(ascending=False)
nan_pct = df.isnull().sum() / len(df) * 100
print(nan_pct.sort_values(ascending=False))
cols_to_drop = nan_pct[nan_pct > 50].index
df = df.drop(columns=cols_to_drop)
df.columns

Schedule                  91.317756
Broad.phase.of.flight     85.689717
Air.carrier               54.430079
Airport.Code              32.736330
Airport.Name              32.083976
Report.Status             18.426047
Engine.Type               15.294746
Total.Serious.Injuries    14.482268
Total.Fatal.Injuries      13.930732
Total.Minor.Injuries      12.958131
Purpose.of.flight         12.032974
Weather.Condition         10.093702
Number.of.Engines          9.411695
Longitude                  7.537659
Latitude                   7.513937
Publication.Date           3.611671
Total.Uninjured            3.504922
Injury.Rate                2.152770
Injury.Severity            1.850314
FAR.Description            1.209821
Registration.Number        0.913296
Amateur.Built              0.059305
Location                   0.017791
Country                    0.005930
Accident.Number            0.000000
Event.Id                   0.000000
Investigation.Type         0.000000
Model                      0

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Report.Status', 'Publication.Date', 'Total.Passengers', 'Injury.Rate',
       'Is.Destroyed', 'Make.Model'],
      dtype='object')

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [10]:
df.to_csv("data/AviationData_cleaned.csv", index=False)